In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain.tools.retriever import create_retriever_tool
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HUGGINGFACEHUB_API_TOKEN"] = str(os.getenv("HUGGINGFACE_API_KEY"))
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = str(os.getenv("LANGCHAIN_API_KEY"))
os.environ["GROQ_API_KEY"] = str(os.getenv("Groq_Api_key"))
# Load documents

loader1 = WebBaseLoader("https://python.langchain.com/docs/tutorials/agents/")
loader2 = WebBaseLoader("https://python.langchain.com/docs/tutorials/llm_chain")
loader3 = WebBaseLoader("https://python.langchain.com/docs/tutorials/rag/")

docs1 = loader1.load_and_split()
docs2 = loader2.load_and_split()
docs3 = loader3.load_and_split()



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:

splitter = RecursiveCharacterTextSplitter(chunk_size = 1000 , chunk_overlap = 100)
splits = splitter.split_documents(docs3)

# Embeddings

embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Vectorstore

index = FAISS.from_documents( splits[:50] ,embed)


c:\codehub\Rag Projects\project 2\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [4]:

#retrieve

retreival = index.as_retriever()

#create retreival tool 

retrieval_tool = create_retriever_tool(retreival, "LangChain Retriever Tool" , 
                                       "answer the questions asking about langchain mindfuly and euthentically")
retrieval_tool.name

#create wikipedia tool

wiki_api_rapper = WikipediaAPIWrapper(top_k_results=1 , doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api_rapper)
wiki_tool.name

# create arxiv tool

arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)
arxiv_tool.name

#combine tools

tools = [arxiv_tool , wiki_tool , retrieval_tool]

#llm 
from langchain_groq import ChatGroq

llm = ChatGroq(model = "llama-3.1-70b-versatile")


In [ ]:

#create agent 

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executer = AgentExecutor(agent=agent,tools=tools)

agent_executer.invoke({'input':'?'})

{'input': 'give the list of pakistan cricket team best playing eleven in 2024?',
 'output': 'The information you are looking for is not available.'}